<a href="https://colab.research.google.com/github/alekriley/alekriley.github.io/blob/master/importance_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primer on Basic MonteCarlo Method
We can approximate the function of a random variable using an average of the function computed over all samples.
> $ E[f(x)]\approx\frac{1}{M}\sum_{m=1}^M f(x^{(m)})$ for samples $x^{(1)},x^{(2)},\dots,x^{(M)}$

In the same way we can approximate the variance of $f(x)$.
>  $ Var[f(x)]\approx\frac{1}{M-1}\sum_{m=1}^M [f(x^{(m)})-u_f]^2$ for samples $x^{(1)},x^{(2)},\dots,x^{(M)}$ with $u_f = \hat E[f(x)]$.

# The Idea of Importance Sampling

Consider the expectation of any function of random variable.

> $$E[f(x)]=\int_x f(x)p(x)dx$$ 

where $p(x)$ is the probability density function corresponding to the random variable x. 
That is, we are taking the expectation with respect to $p(x)$. Notice however that we can easily do the following.
> $$E[f(x)]=\int_x q(x)f(x)\frac{p(x)}{q(x)}dx=E_{q(x)}\left[f(x)\frac{p(x)}{q(x)}\right]$$

We can express the expectation of $f(x)$ with respect to a probability distribution $q(x)$.